In [2]:
#import libraries
import pandas
import scipy 
import numpy as np


In [3]:
#dataset link
url = "https://people.sc.fsu.edu/~jburkardt/datasets/regression/x28.txt"
names = ['I', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'B'] 
dataframe = pandas.read_csv(url, delimiter=r"\s+", header=71, names=names)
X = dataframe.iloc[:, 1:-1]
Y = dataframe.iloc[:, -1]
X.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
0,36,27,71,8.1,3.34,11.4,81.5,3243,8.8,42.6,11.7,21,15,59,59
1,35,23,72,11.1,3.14,11.0,78.8,4281,3.6,50.7,14.4,8,10,39,57
2,44,29,74,10.4,3.21,9.8,81.6,4260,0.8,39.4,12.4,6,6,33,54
3,47,45,79,6.5,3.41,11.1,77.5,3125,27.1,50.2,20.6,18,8,24,56
4,43,35,77,7.6,3.44,9.6,84.6,6441,24.4,43.7,14.3,43,38,206,55


In [4]:
#data normalization
def normalize_and_add_ones(X):
  X = np.array(X)
  
  X_normalized = (X - np.min(X, axis=0))/(np.max(X, axis=0) - np.min(X, axis=0))
  
  return np.concatenate((np.ones(X.shape[0]).reshape(-1,1), X_normalized), axis=1)

In [5]:
#model building
class RidgeRegression:
  def __init__(self):
    return
  def fit(self, X_train, Y_train, LAMBDA):
    assert len(X_train.shape) == 2 and X_train.shape[0] == Y_train.shape[0]

    w = np.linalg.pinv(np.dot(X_train.T,X_train)+LAMBDA*np.eye(X_train.shape[1])).dot(np.dot(X_train.T, Y_train))
    
    return w

  def fit_gradient_descent(self, X_train, Y_train, LAMBDA, learning_rate, max_epoch=100, batch_size=128):
    w = np.random.randn(X_train.shape[1])
    last_loss = 1e+9

    for ep in range(max_epoch):
      shuffle_idx = np.random.shuffle(np.arange(X_train.shape[0]))
      X_train = X_train[shuffle_idx]
      Y_train = Y_train[shuffle_idx]
      total_minibatch = int(np.ceil(X_train.shape[0]/batch_size))
      for i in range(total_minibatch):
        index = i*batch_size
        X_train_sub = X_train[index:index+batch_size]
        Y_train_sub = Y_train[index:index+batch_size]
        grad = X_train_sub.T.dot(np.dot(X_train_sub, w)-Y_train_sub) + LAMBDA*w
        w = w - learning_rate*grad
      new_loss = self.compute_RSS(self.predict(w, X_train), Y_train)
      if abs(new_loss-last_loss) <= 1e-5:
        break
      last_loss = new_loss
    return w
    
  def predict(self, W, X_new):
    X_new = np.array(X_new)
    Y_new = X_new.dot(W)
    return Y_new
  def compute_RSS(self, Y_new, Y_predicted):
    loss = 1. / Y_new.shape[0] * \
            np.sum((Y_new - Y_predicted) ** 2)
    return loss
  def get_the_best_LAMBDA(self, X_train, Y_train):
    def cross_validation(num_folds, LAMBDA):
      row_ids = np.arange(X_train.shape[0])

      #np.split() requires equal divisions
      valid_ids = np.split(row_ids[:len(row_ids) - len(row_ids) % num_folds], num_folds)
      valid_ids[-1] = np.append(valid_ids[-1], row_ids[len(row_ids) - len(row_ids) % num_folds:])
      train_ids = [[k for k in row_ids if k not in valid_ids[i]] for i in range(num_folds)]
      aver_RSS = 0

      for i in range(num_folds):
        valid_part = {'X': X_train[valid_ids[i]], 'Y': Y_train[valid_ids[i]]}
        train_part = {'X': X_train[train_ids[i]], 'Y': Y_train[train_ids[i]]}
        W = self.fit(train_part['X'], train_part['Y'], LAMBDA)
        Y_predicted = self.predict(W, valid_part['X'])
        aver_RSS += self.compute_RSS(valid_part['Y'], Y_predicted)
      return aver_RSS / num_folds
      
    def range_scan(best_LAMBDA, minimum_RSS, LAMBDA_values):
      for current_LAMBDA in LAMBDA_values:
        aver_RSS = cross_validation(num_folds=5, LAMBDA=current_LAMBDA)
        if aver_RSS < minimum_RSS:
          best_LAMBDA = current_LAMBDA
          minimum_RSS = aver_RSS
      return best_LAMBDA, minimum_RSS
    
    best_LAMBDA, minimum_RSS = range_scan(best_LAMBDA=0, minimum_RSS=1e8, LAMBDA_values=range(50))

    LAMBDA_values = [k * 1. / 1000 for k in range(
        max(0, (best_LAMBDA-1) * 1000), (best_LAMBDA + 1) * 1000, 1)
                    ]
    
    best_LAMBDA, minimum_RSS = range_scan(best_LAMBDA=best_LAMBDA, minimum_RSS=minimum_RSS, LAMBDA_values=LAMBDA_values)

    return best_LAMBDA



In [6]:
if __name__ == "__main__":
  Y = Y.to_numpy()
  # normalization
  X = normalize_and_add_ones(X)
  
  X_train, Y_train = X[:50], Y[:50]
  X_test, Y_test = X[50:], Y[50:]

  ridge_regression = RidgeRegression()
  best_LAMBDA = ridge_regression.get_the_best_LAMBDA(X_train, Y_train)
  print('best lambda:', best_LAMBDA)
  W_learned = ridge_regression.fit(X_train=X_train, Y_train=Y_train, LAMBDA=best_LAMBDA)
  Y_predicted = ridge_regression.predict(W=W_learned, X_new=X_test)

  print(ridge_regression.compute_RSS(Y_new=Y_test, Y_predicted=Y_predicted))

best lambda: 0.002
1527.069807804805
